# Dependencies and data

In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error as mse
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor

from joblib import dump, load, Parallel, delayed

In [2]:
metadata = pd.read_csv('../data/metadata.csv')
sub_format = pd.read_csv('../data/submission_format.csv')
train_labels = pd.read_csv('../data/train_labels.csv')

In [3]:
metadata.date = pd.to_datetime(metadata.date)
metadata['year'] = metadata.date.dt.year
metadata['month'] = metadata.date.dt.month
metadata['week'] = metadata.date.dt.isocalendar().week

seasons = {
    1: 1,
    2: 1,
    3: 2,
    4: 2,
    5: 2,
    6: 3,
    7: 3,
    8: 3,
    9: 4,
    10: 4,
    11: 4,
    12: 1
}

reg_sev_map = {
    'midwest': 2,
    'northeast': 2,
    'south' : 2,
    'west' : 4
}

reg_map = {
    'south' : 0,
    'northeast' : 1,
    'west' : 2,
    'midwest' : 3
}

metadata['season'] = metadata.month.map(seasons)

region = pd.concat((train_labels, sub_format[['region', 'uid']]), axis=0)

all_data = pd.merge(metadata, region, on='uid', how='left')
data = all_data.copy(deep=True)
data

,uid,latitude,longitude,date,split,year,month,week,season,region,severity,density
0,aabm,39.080319,-86.430867,2018-05-14,train,2018,5,20,2,midwest,1.0,585.0
1,aabn,36.559700,-121.510000,2016-08-31,test,2016,8,35,3,west,NaN,NaN
2,aacd,35.875083,-78.878434,2020-11-19,train,2020,11,47,4,south,1.0,290.0
3,aaee,35.487000,-79.062133,2016-08-24,train,2016,8,34,3,south,1.0,1614.0
4,aaff,38.049471,-99.827001,2019-07-23,train,2019,7,30,3,midwest,3.0,111825.0
...,...,...,...,...,...,...,...,...,...,...,...,...
23565,zzvv,36.708500,-121.749000,2014-12-02,test,2014,12,49,1,west,NaN,NaN
23566,zzwo,39.792190,-99.971050,2017-06-19,train,2017,6,25,3,midwest,2.0,48510.0
23567,zzwq,35.794000,-79.012551,2015-03-24,train,2015,3,13,2,south,1.0,1271.0
23568,zzyb,35.742000,-79.238600,2016-11-21,train,2016,11,47,4,south,1.0,9682.0


# Utils

In [4]:

def analyize_matches(y_true, y_pred, plot=False):
    print("Exact matches: ", sum(y_true == y_pred) / len(y_true))
    print("Missed by 1: ", sum(abs(y_true - y_pred) == 1) / len(y_true))
    print("Missed by 2: ", sum(abs(y_true - y_pred) == 2) / len(y_true))
    print("Missed by 3: ", sum(abs(y_true - y_pred) == 3) / len(y_true))
    print("Missed by 4: ", sum(abs(y_true - y_pred) == 4) / len(y_true))
    
    stupid_vals = []
    for i in range(1, 6):
        stupid_vals.append(
            ((sum([1 for x, y in zip(y_true, y_pred) if x == i and y == i])/len(y_true))*100, (sum(y_true == i)/len(y_true))*100)
            )

    print()
    for i in range(5):
        print(f"Severity {i+1} : accuracy: {np.round(stupid_vals[i][0], 3)} % - prevalence: {np.round(stupid_vals[i][1], 3)} %")
    
    try:
        print()
        print("Classification report:")
        print(classification_report(y_true, y_pred))
    except Exception as e:
        print(e)
        print("Classification report failed")
        
    if plot:
        print()
        sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, fmt='d', cmap='Reds')


def rmse(y_true, y_pred):
    return mse(y_true, y_pred, squared=False)

def get_data_by_date( date=None, data=None):
    return data[data.date == date]


def get_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Knn fts

In [5]:
def knn(row=None, train_data=None, k=1):
    """
    row : pd.Series (row from val_data)
    train_data : pd.DataFrame (all_data)
    k : int number of nearest neighbours to consider
    
    algo:
    1. Get past month data collected till the current row
    2. Get the k nearest neighbours (geodesic dist using lat, lng) from the above data
    3. Get the mean of the severity from the above rows
    4. Return the mean of the nearest neighbours severity

    """
    
    if row is None:
        print('Row None bruv!')
        return None
    
    uid = row.uid
    date = row.date
    region = row.region
    past_date = date - pd.Timedelta(days=30)
    
    past_month_data = train_data[(train_data.date < date) & (train_data.date >= past_date)]
    past_month_data.sort_values(by='date', inplace=True)
    
    # if no past data, return the mean of the region
    if len(past_month_data) == 0:
        return reg_sev_map[region]
        
    dist_matrix =pd.DataFrame(columns=['uid', 'dist'])       # 0th col for uid, 1st col for dist
    for i, past_row in enumerate(past_month_data.itertuples()):
        dist_matrix.loc[i, 'uid'] = past_row.uid
        dist_matrix.loc[i, 'dist'] = get_distance(row.latitude, row.longitude, past_row.latitude, past_row.longitude)   # returns geodesic dist in km

    # get mean of top k nearest neighbours
    n_uids = dist_matrix.sort_values(by='dist').head(k).uid.values
    nn_severity = train_data[train_data.uid.isin(n_uids)].severity.mean()
    
    return np.round(nn_severity)

In [7]:
# def knn_wrapper(data, k=5):
#     sev_list = Parallel(n_jobs=-1, backend='loky')([delayed(knn)(row, train_data=data, k=k) for row in data.itertuples()])
#     return sev_list

In [6]:
data = data.sort_values(by='date')
data

,uid,latitude,longitude,date,split,year,month,week,season,region,severity,density
4387,evep,44.847993,-93.476318,2013-01-04,train,2013,1,1,1,midwest,1.0,115.0
13644,paev,44.822478,-93.367962,2013-01-04,train,2013,1,1,1,midwest,1.0,1884.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,2013,1,1,1,midwest,1.0,1416.0
6144,guny,44.878889,-93.490833,2013-01-04,train,2013,1,1,1,midwest,1.0,558.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,2013,1,1,1,midwest,1.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
17559,thki,36.725400,-121.730000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
17452,teuu,36.772300,-121.788000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
14254,prfi,36.751800,-121.742000,2021-12-29,test,2021,12,52,1,west,NaN,NaN


In [8]:
# knn5_preds = []
# for row in tqdm(data.itertuples(), total=data.shape[0]):
#     severity = knn(row, train_data=data, k=5)
#     knn5_preds.append(severity)

  2%|▏         | 543/23570 [00:37<26:33, 14.45it/s]


KeyboardInterrupt: 

# Xgboost

In [108]:
data = data.sort_values(by='date')
data

,uid,latitude,longitude,date,split,year,month,week,season,region,severity,density
4387,evep,44.847993,-93.476318,2013-01-04,train,2013,1,1,1,midwest,1.0,115.0
13644,paev,44.822478,-93.367962,2013-01-04,train,2013,1,1,1,midwest,1.0,1884.0
5566,gdxr,44.877646,-93.557842,2013-01-04,train,2013,1,1,1,midwest,1.0,1416.0
6144,guny,44.878889,-93.490833,2013-01-04,train,2013,1,1,1,midwest,1.0,558.0
5317,fwbt,44.850500,-93.515700,2013-01-04,train,2013,1,1,1,midwest,1.0,476.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12443,nsoi,36.736800,-121.734000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
17559,thki,36.725400,-121.730000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
17452,teuu,36.772300,-121.788000,2021-12-29,test,2021,12,52,1,west,NaN,NaN
14254,prfi,36.751800,-121.742000,2021-12-29,test,2021,12,52,1,west,NaN,NaN


In [109]:
train, test = train_test_split(data[data.split == 'train'], test_size=0.05, random_state=42, shuffle=True)
train.shape, test.shape

((16207, 12), (853, 12))

In [110]:
train.region = train.region.map(reg_map)
test.region = test.region.map(reg_map)

train.week = train.week.astype('int')
test.week = test.week.astype('int')

In [111]:
drop_cols = ['uid', 'split', 'date', 'severity', 'density']

X_train = train.drop(drop_cols, axis=1)
y_train = train['severity']

X_val = test.drop(drop_cols, axis=1)
y_val = test['severity']

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((16207, 7), (16207,), (853, 7), (853,))

In [112]:
y_train.value_counts(normalize=True), y_val.value_counts(normalize=True)

(1.0    0.440427
 4.0    0.207811
 2.0    0.189609
 3.0    0.158697
 5.0    0.003455
 Name: severity, dtype: float64,
 1.0    0.420868
 4.0    0.209848
 2.0    0.194607
 3.0    0.172333
 5.0    0.002345
 Name: severity, dtype: float64)

In [113]:
X_val

,latitude,longitude,year,month,week,season,region
11355,37.539400,-121.122000,2019,12,49,1,2
12083,35.742000,-79.201565,2015,3,13,2,0
5836,44.140000,-87.233333,2019,4,14,2,3
16401,35.980000,-78.814305,2019,11,47,4,0
12240,44.850500,-93.515700,2013,7,29,3,3
...,...,...,...,...,...,...,...
6862,35.680000,-79.109342,2021,7,28,3,0
1237,35.866170,-78.807097,2015,12,50,1,0
1131,35.858654,-78.746945,2019,7,27,3,0
21508,38.367700,-121.521000,2017,5,20,2,2


In [164]:
%%time

xgb_reg = XGBRegressor(
    n_estimators=2000, 
    max_depth=5, 
    learning_rate=0.02, 
    subsample=1,
    colsample_bytree=0.8,
    grow_policy='depthwise',
    sampling_method='uniform',
    objective='reg:squarederror', 
    tree_method='gpu_hist',
    gpu_id=0, 
    n_jobs=-1, 
    verbose=1, 
    random_state=42
    )

xgb_reg.fit(X_train, y_train)
preds = xgb_reg.predict(X_val)
preds = pd.Series(np.round(preds)).clip(1, 5).values
print("train rmse", rmse(y_train, xgb_reg.predict(X_train)))
print("test rmse:", rmse(y_val, preds))

[23:03:33] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


train rmse 0.5655955164206753
test rmse: 0.6787686251031234
CPU times: total: 3.72 s
Wall time: 6.71 s


In [147]:
analyize_matches(y_val, preds)

Exact matches:  0.6412661195779601
Missed by 1:  0.3141852286049238
Missed by 2:  0.044548651817116064
Missed by 3:  0.0
Missed by 4:  0.0

Severity 1 : accuracy: 28.37 % - prevalence: 42.087 %
Severity 2 : accuracy: 12.309 % - prevalence: 19.461 %
Severity 3 : accuracy: 5.041 % - prevalence: 17.233 %
Severity 4 : accuracy: 18.406 % - prevalence: 20.985 %
Severity 5 : accuracy: 0.0 % - prevalence: 0.234 %

Classification report:
              precision    recall  f1-score   support

         1.0       0.80      0.67      0.73       359
         2.0       0.35      0.63      0.45       166
         3.0       0.54      0.29      0.38       147
         4.0       0.91      0.88      0.89       179
         5.0       0.00      0.00      0.00         2

    accuracy                           0.64       853
   macro avg       0.52      0.50      0.49       853
weighted avg       0.69      0.64      0.65       853



In [69]:
X_val.isin(X_train).all()

latitude     False
longitude    False
year         False
month        False
week         False
season       False
region       False
dtype: bool

In [188]:
%%time

lgb_reg = LGBMRegressor(
    n_estimators=2000, 
    max_depth=5, 
    learning_rate=0.02, 
    random_state=42,
    num_leaves=2**5,
    n_jobs=-1,
    device_type='cpu',
    )

lgb_reg.fit(X_train, y_train)
preds = lgb_reg.predict(X_val)
preds = pd.Series(np.round(preds)).clip(1, 5).values
print("train rmse", rmse(y_train, lgb_reg.predict(X_train)))
print("test rmse:", rmse(y_val, preds))

train rmse 0.5795191932025531
test rmse: 0.6813544197053774
CPU times: total: 11.7 s
Wall time: 2.02 s


In [203]:
%%time

hgb_reg = HistGradientBoostingRegressor(random_state=42, learning_rate=0.05, max_iter=300, early_stopping=True, scoring='loss')
hgb_reg.fit(X_train, y_train)
preds = hgb_reg.predict(X_val)
preds = pd.Series(np.round(preds)).clip(1, 5).values
print("train rmse", rmse(y_train, hgb_reg.predict(X_train)))
print("test rmse:", rmse(y_val, preds))
print()
analyize_matches(y_val, preds)

train rmse 0.6029838488744201
test rmse: 0.6958257722127704

Exact matches:  0.6459554513481829
Missed by 1:  0.3106682297772567
Missed by 2:  0.04337631887456037
Missed by 3:  0.0
Missed by 4:  0.0

Severity 1 : accuracy: 28.488 % - prevalence: 42.087 %
Severity 2 : accuracy: 12.427 % - prevalence: 19.461 %
Severity 3 : accuracy: 5.393 % - prevalence: 17.233 %
Severity 4 : accuracy: 18.288 % - prevalence: 20.985 %
Severity 5 : accuracy: 0.0 % - prevalence: 0.234 %

Classification report:
              precision    recall  f1-score   support

         1.0       0.80      0.68      0.73       359
         2.0       0.36      0.64      0.46       166
         3.0       0.56      0.31      0.40       147
         4.0       0.91      0.87      0.89       179
         5.0       0.00      0.00      0.00         2

    accuracy                           0.65       853
   macro avg       0.53      0.50      0.50       853
weighted avg       0.69      0.65      0.65       853

CPU times: total:

# Test predictions

In [204]:
# what the fucking fuck?!!!!

all_train = all_data[all_data.split == 'train']
all_test = all_data[all_data.split == 'test']

all_train.region = all_train.region.map(reg_map)
all_test.region = all_test.region.map(reg_map)

all_train.week = all_train.week.astype('int')
all_test.week = all_test.week.astype('int')

drop_cols = ['uid', 'split', 'date', 'severity', 'density']
all_train.shape, all_test.shape

((17060, 12), (6510, 12))

In [205]:
X = all_train.drop(drop_cols, axis=1)
y = all_train['severity']
X_test = all_test.drop(drop_cols, axis=1)

X.shape, y.shape, X_test.shape

((17060, 7), (17060,), (6510, 7))

In [206]:
hgb_reg = HistGradientBoostingRegressor(random_state=42, learning_rate=0.05, max_iter=300, early_stopping=True, scoring='loss')
hgb_reg.fit(X, y)
test_preds = hgb_reg.predict(X_test)
test_preds = pd.Series(np.round(test_preds)).clip(1, 5).values
print("train rmse", rmse(y, hgb_reg.predict(X)))

train rmse 0.5917295419527717


In [207]:
y.value_counts(normalize=True)

1.0    0.439449
4.0    0.207913
2.0    0.189859
3.0    0.159379
5.0    0.003400
Name: severity, dtype: float64

In [208]:
pd.Series(test_preds).value_counts(normalize=True)

2.0    0.390015
1.0    0.277573
4.0    0.208141
3.0    0.124270
dtype: float64

# Submission

In [209]:
sub_format.severity = test_preds.astype(int)
display(sub_format.sample(5))
sub_format.to_csv('../submissions/to submit/idkanything_histgrad.csv', index=False)

,uid,region,severity
6329,ziou,midwest,3
5274,vblu,northeast,2
4418,rsos,west,2
213,axlb,northeast,2
5977,xxxb,west,4
